<a href="https://colab.research.google.com/github/Elizav555/Data_Mining/blob/main/Distances/Distances.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import nltk
import requests
import string
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Сначала я решила взять статью https://www.thegamer.com/horizon-forbidden-west-review-ps5/, которую трудно отнести к какой-либо категории, но мне был интересен результат 

In [5]:
url = 'https://www.thegamer.com/horizon-forbidden-west-review-ps5/'
headers = requests.utils.default_headers()
headers.update(
    {
        'User-Agent': 'My User Agent 1.0',
    }
)
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text)
root = soup.body
root_childs = [e.name for e in root.descendants if e.name is not None]
#print(set(root_childs))

В коде выше я просматривала список Html тэгов, которые содержат текст. Ниже я получила все слова, которые они содержат

In [26]:
textTags = soup.find_all(['p','h1','h2','h3','h4','label','a','ul','li'])
whole_text = ' '.join(map(lambda it: it.text, textTags)) 
all_words = word_tokenize(whole_text)
#print(all_words)
print('Number of all words: ',len(all_words))

Number of all words:  2636


Функцию ниже я использовала, чтобы очищать текст от знаков пунктуации, стоп слов и цифр как при обработке статей, так и при составлении условных словарей, которые содержали слова, относящиеся к категориям

In [41]:
def process_whole_text(all_words_set):
  symb_to_remove = stopwords.words('english') +  list(string.punctuation) +  list(string.digits) + list('’')
  text_cleared = [word for word in all_words_set if word not in symb_to_remove]
  text_cleared = [word for word in text_cleared if word.isalpha()]
  return text_cleared

horizon_article = process_whole_text(all_words)
print('Number of cleared words: ',len(horizon_article))

Number of cleared words:  1599


https://keywordgenerator.net

Сервис, откуда я взяла слова для топиков

In [42]:
def process_key_words(file_name, key ):
  dataset = pd.read_csv(file_name)
  dataset = dataset[key]
  words = dataset.values.tolist()
  words = list(map(lambda it: it.split(' ') ,words))
  words_set = list()
  for words in words:
    words_set+=words
  words_set = set(words_set)
  words_set-=set(['']) 
  words_set = process_whole_text(words_set)
  return set(words_set)

In [43]:
news_words = process_key_words('news.csv','Keyword')
science_words = process_key_words('science.csv','Keyword')
shopping_words = process_key_words('shopping.csv','Keyword')
sport_words = process_key_words('sport.csv','Keyword')

news_words = news_words.union(process_key_words('news2.csv','Keywords'))
science_words = science_words.union(process_key_words('science2.csv','Keywords'))
shopping_words = shopping_words.union(process_key_words('shopping2.csv','Keywords'))
sport_words = sport_words.union(process_key_words('sport2.csv','Keywords'))
print('News: ', len(news_words))
print('Science: ', len(science_words))
print('Shopping: ', len(shopping_words))
print('Sport: ', len(sport_words))

News:  939
Science:  1034
Shopping:  1003
Sport:  1008


In [44]:
def parse_from_dictionary(url):
  response_dictionary = requests.get(url)
  soup_dictionary = BeautifulSoup(response_dictionary.text)
  textTags_dictionary = soup_dictionary.find_all('a',class_='btnpopover')
  whole_text_dictionary = ' '.join(map(lambda it: it.text, textTags_dictionary)) 
  all_words_dictionary = word_tokenize(whole_text_dictionary)
  all_words_dictionary = process_whole_text(all_words_dictionary)
  return(set(all_words_dictionary))


https://www.niftyword.com/related-words/science/ связанные слова

In [45]:
news_words = news_words.union(parse_from_dictionary('https://www.niftyword.com/related-words/news/'))
science_words = science_words.union(parse_from_dictionary('https://www.niftyword.com/related-words/science/'))
shopping_words= shopping_words.union(parse_from_dictionary('https://www.niftyword.com/related-words/shopping/'))
sport_words =sport_words.union(parse_from_dictionary('https://www.niftyword.com/related-words/sport/'))
print('News: ', len(news_words))
print('Science: ', len(science_words))
print('Shopping: ', len(shopping_words))
print('Sport: ', len(sport_words))

News:  2710
Science:  3611
Shopping:  3640
Sport:  3358


https://keywordtool.io/ru/search/keywords/google/

еще один сервис, в котором искала ключевые слова, теперь я могла приступать к анализу первой статьи

In [46]:
def jaccard_fun(a,b,topic):
  a= set(a)
  b=set(b)
  shared = a.intersection(b)
  total = a.union(b)
  res = len(shared) / len(total)
  print('Topic: ',topic,', Shared: ',len(shared),', Total: ',len(total),', Similarity: ',res,', Distance: ',1-res)

In [47]:
jaccard_fun(horizon_article ,news_words,'News')
jaccard_fun(horizon_article ,science_words,'Science')
jaccard_fun(horizon_article ,shopping_words,'Shopping')
jaccard_fun(horizon_article ,sport_words,'Sport')

Topic:  News , Shared:  97 , Total:  3326 , Similarity:  0.02916416115453999 , Distance:  0.97083583884546
Topic:  Science , Shared:  116 , Total:  4208 , Similarity:  0.027566539923954372 , Distance:  0.9724334600760456
Topic:  Shopping , Shared:  121 , Total:  4232 , Similarity:  0.028591682419659734 , Distance:  0.9714083175803403
Topic:  Sport , Shared:  126 , Total:  3945 , Similarity:  0.03193916349809886 , Distance:  0.9680608365019011


Первая выбранная мною статья, оцененная с помощью **коэф жаккара** **оказалась на примерно равном расстоянии** от всех категорий - **приблизительно 0,97**. Однако коэфициент схожести для категорий **новости** и **спорт** были чуть выше, что я считаю верным, так как материал статьи это ревью на новую компьютерную игру

https://www.topendsports.com/world/lists/popular-sport/ - статья про популярные виды спорта оказалась моей следующей жертвой

In [48]:
def process_article(link):
  response_dictionary = requests.get(link)
  soup_dictionary = BeautifulSoup(response_dictionary.text)
  textTags_dictionary = soup_dictionary.find_all(['p','h1','li','a','h2'])
  whole_text_dictionary = ' '.join(map(lambda it: it.text, textTags_dictionary)) 
  all_words_dictionary = word_tokenize(whole_text_dictionary)
  #print(all_words_dictionary)
  print('Number of all words: ',len(all_words_dictionary))
  article = process_whole_text(all_words_dictionary)
  print('Number of cleared words: ',len(article))
  return article

In [49]:
sport_article = process_article('https://www.topendsports.com/world/lists/popular-sport/')

Number of all words:  16225
Number of cleared words:  7915


In [50]:
jaccard_fun(sport_article ,news_words,'News')
jaccard_fun(sport_article ,science_words,'Science')
jaccard_fun(sport_article ,shopping_words,'Shopping')
jaccard_fun(sport_article ,sport_words,'Sport')

Topic:  News , Shared:  337 , Total:  5059 , Similarity:  0.06661395532713975 , Distance:  0.9333860446728602
Topic:  Science , Shared:  401 , Total:  5896 , Similarity:  0.06801221166892808 , Distance:  0.9319877883310719
Topic:  Shopping , Shared:  431 , Total:  5895 , Similarity:  0.0731128074639525 , Distance:  0.9268871925360475
Topic:  Sport , Shared:  509 , Total:  5535 , Similarity:  0.09196025293586269 , Distance:  0.9080397470641373


Здесь **коэф жаккара** определил, что наименьшее растояние между этой статьей и тематикой **спорта** **0,9** , однако из-за большой разницы в размерых словарей и кол-ва слов из статьи схожесть все еще невелика.

In [51]:
science_article = process_article('https://en.wikipedia.org/wiki/Science')
jaccard_fun(science_article ,news_words,'News')
jaccard_fun(science_article ,science_words,'Science')
jaccard_fun(science_article ,shopping_words,'Shopping')
jaccard_fun(science_article ,sport_words,'Sport')

Number of all words:  32844
Number of cleared words:  15550
Topic:  News , Shared:  470 , Total:  6768 , Similarity:  0.06944444444444445 , Distance:  0.9305555555555556
Topic:  Science , Shared:  862 , Total:  7277 , Similarity:  0.11845540744812423 , Distance:  0.8815445925518758
Topic:  Shopping , Shared:  546 , Total:  7622 , Similarity:  0.07163474153765416 , Distance:  0.9283652584623459
Topic:  Sport , Shared:  530 , Total:  7356 , Similarity:  0.07205002718868951 , Distance:  0.9279499728113105


Тогда я решила проверить, что даст анализ первой страницы, которая выходит,когда гуглишь science - любимая википедия. Расстояние до категории **науки** ожидаемо **наименьшее и равно 0,88** , до остальных тем примерно одинаково и равно 0,93

In [52]:
data = [" ".join(horizon_article), " ".join(sport_article)," ".join(science_article)," ".join(news_words)," ".join(science_words)," ".join(shopping_words)," ".join(sport_words)]
count_vectorizer = CountVectorizer()
vector_matrix = count_vectorizer.fit_transform(data)
tokens = count_vectorizer.get_feature_names_out()
tokens

array(['aa', 'aaa', 'aaboe', ..., 'ꮳꮃꭹ', '한국어', 'ﬁrst'], dtype=object)

In [53]:
vector_matrix_array = vector_matrix.toarray()
vector_matrix_array

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 2, 2, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [54]:
def create_dataframe(matrix, tokens):
    doc_names = ["horizon_article","sport_article","science_article","news_words","science_words","shopping_words","sport_words"]
    df = pd.DataFrame(data=matrix, index=doc_names, columns=tokens)
    return(df)

dataframe = create_dataframe(vector_matrix_array,tokens)
dataframe

,aa,aaa,aaboe,aadvantage,aaj,aakash,aand,abandon,abandoned,abandonment,...,太史公書,孫叔敖,文言,日本語,粵語,西門豹,贛語,ꮳꮃꭹ,한국어,ﬁrst
horizon_article,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
sport_article,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
science_article,0,0,1,0,0,0,0,0,0,0,...,1,2,2,2,2,1,2,2,2,1
news_words,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
science_words,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
shopping_words,1,0,0,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
sport_words,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


Тогда я решила попробовать представить свои сеты данных в виде векторов, где 1 обозначает, что данное слово присутствует в наборе (больше 1 если несколько раз), а 0 при его отсутствии

In [56]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity_matrix = cosine_similarity(vector_matrix)
create_dataframe(cosine_similarity_matrix,["horizon_article","sport_article","science_article","news_words","science_words","shopping_words","sport_words"])

,horizon_article,sport_article,science_article,news_words,science_words,shopping_words,sport_words
horizon_article,1.000000,0.263151,0.205300,0.071757,0.067608,0.075520,0.077928
sport_article,0.263151,1.000000,0.098645,0.078464,0.077189,0.077457,0.119270
science_article,0.205300,0.098645,1.000000,0.104413,0.152270,0.085043,0.103770
news_words,0.071757,0.078464,0.104413,1.000000,0.195690,0.285399,0.189575
science_words,0.067608,0.077189,0.152270,0.195690,1.000000,0.243459,0.274859
shopping_words,0.075520,0.077457,0.085043,0.285399,0.243459,1.000000,0.345189
sport_words,0.077928,0.119270,0.103770,0.189575,0.274859,0.345189,1.000000


Метрики **косинуса** дали результаты, которые представлены выше. В целом также как и для коэф жаккара **для статьи обзора игры** дистанция до тематикb **спорт** оказалось наименьшим и примерно равным 0,92 , однако следующей по близости идет категория **шоппинг**, а не новости

Для статьи про спорт результаты оказались **более показательными**, так как разница в схожести между спортом и остальными категориями увеличилась: 
**для спорта расстояние примерно 0,88** 

Вики статья же показывает наибольшие средние результаты из-за своего размера. Наиближайшей темой верно оказывается **наука 0,848**, далее идут категории **новости и спорт** с примерным расстоянием **0,89**

In [57]:
def jaccard_vector_fun(x, y,topic):
    intersect = 0
    union = 0
    for i,val in enumerate(x):
      if x[i]&y[i]!=0:
        intersect +=1
      if x[i]|y[i]!=0:
        union +=1
    res = intersect / union
    print('Topic: ',topic,', Shared: ',intersect,', Total: ',union,', Similarity: ',res,', Distance: ',1-res)

In [58]:
jaccard_vector_fun(vector_matrix_array[0] ,vector_matrix_array[3],'News')
jaccard_vector_fun(vector_matrix_array[0] ,vector_matrix_array[4],'Science')
jaccard_vector_fun(vector_matrix_array[0],vector_matrix_array[5],'Shopping')
jaccard_vector_fun(vector_matrix_array[0],vector_matrix_array[6],'Sport')

Topic:  News , Shared:  90 , Total:  3231 , Similarity:  0.027855153203342618 , Distance:  0.9721448467966574
Topic:  Science , Shared:  96 , Total:  4109 , Similarity:  0.023363348746653686 , Distance:  0.9766366512533463
Topic:  Shopping , Shared:  113 , Total:  4128 , Similarity:  0.027374031007751938 , Distance:  0.9726259689922481
Topic:  Sport , Shared:  109 , Total:  3841 , Similarity:  0.028378026555584482 , Distance:  0.9716219734444155


In [59]:
jaccard_vector_fun(vector_matrix_array[1] ,vector_matrix_array[3],'News')
jaccard_vector_fun(vector_matrix_array[1] ,vector_matrix_array[4],'Science')
jaccard_vector_fun(vector_matrix_array[1],vector_matrix_array[5],'Shopping')
jaccard_vector_fun(vector_matrix_array[1],vector_matrix_array[6],'Sport')

Topic:  News , Shared:  244 , Total:  4578 , Similarity:  0.05329838357361293 , Distance:  0.9467016164263871
Topic:  Science , Shared:  264 , Total:  5431 , Similarity:  0.048609832443380595 , Distance:  0.9513901675566194
Topic:  Shopping , Shared:  291 , Total:  5430 , Similarity:  0.05359116022099448 , Distance:  0.9464088397790055
Topic:  Sport , Shared:  357 , Total:  5056 , Similarity:  0.07060917721518987 , Distance:  0.9293908227848101


In [60]:
jaccard_vector_fun(vector_matrix_array[2] ,vector_matrix_array[3],'News')
jaccard_vector_fun(vector_matrix_array[2] ,vector_matrix_array[4],'Science')
jaccard_vector_fun(vector_matrix_array[2],vector_matrix_array[5],'Shopping')
jaccard_vector_fun(vector_matrix_array[2],vector_matrix_array[6],'Sport')

Topic:  News , Shared:  341 , Total:  6123 , Similarity:  0.05569165441776907 , Distance:  0.944308345582231
Topic:  Science , Shared:  604 , Total:  6534 , Similarity:  0.09243954698500154 , Distance:  0.9075604530149984
Topic:  Shopping , Shared:  350 , Total:  7005 , Similarity:  0.049964311206281226 , Distance:  0.9500356887937188
Topic:  Sport , Shared:  356 , Total:  6747 , Similarity:  0.05276419149251519 , Distance:  0.9472358085074848


Выше я проверила, будет ли более эффективным применение **коэф жаккара** для **векторов**, но результаты не сильно изменились